# Modeling with Word2Vec

In [1]:
import pickle
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

Loading in the pickled sentences

In [2]:
with open('../data/sentences.txt', 'rb') as fp:
    sentences = pickle.load(fp)

In [3]:
len(sentences)

795534

Creating some logs of my various parameters, initializing the Word2Vec model, and training on the sentences data. 

In [4]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 10       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# Calling init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# Saving the model to use later
model.save('../data/300features_40minwords_10context')

2019-05-17 13:19:03,094 : INFO : 'pattern' package not found; tag filters are not available for English
2019-05-17 13:19:03,099 : INFO : collecting all words and their counts
2019-05-17 13:19:03,099 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-17 13:19:03,133 : INFO : PROGRESS: at sentence #10000, processed 225799 words, keeping 17773 word types
2019-05-17 13:19:03,169 : INFO : PROGRESS: at sentence #20000, processed 451864 words, keeping 24940 word types
2019-05-17 13:19:03,205 : INFO : PROGRESS: at sentence #30000, processed 671284 words, keeping 30022 word types
2019-05-17 13:19:03,240 : INFO : PROGRESS: at sentence #40000, processed 897762 words, keeping 34337 word types
2019-05-17 13:19:03,275 : INFO : PROGRESS: at sentence #50000, processed 1116945 words, keeping 37748 word types


Training model...


2019-05-17 13:19:03,310 : INFO : PROGRESS: at sentence #60000, processed 1338366 words, keeping 40707 word types
2019-05-17 13:19:03,345 : INFO : PROGRESS: at sentence #70000, processed 1561514 words, keeping 43317 word types
2019-05-17 13:19:03,381 : INFO : PROGRESS: at sentence #80000, processed 1780803 words, keeping 45695 word types
2019-05-17 13:19:03,416 : INFO : PROGRESS: at sentence #90000, processed 2004876 words, keeping 48112 word types
2019-05-17 13:19:03,453 : INFO : PROGRESS: at sentence #100000, processed 2226855 words, keeping 50183 word types
2019-05-17 13:19:03,488 : INFO : PROGRESS: at sentence #110000, processed 2446475 words, keeping 52056 word types
2019-05-17 13:19:03,525 : INFO : PROGRESS: at sentence #120000, processed 2668648 words, keeping 54090 word types
2019-05-17 13:19:03,561 : INFO : PROGRESS: at sentence #130000, processed 2894169 words, keeping 55819 word types
2019-05-17 13:19:03,595 : INFO : PROGRESS: at sentence #140000, processed 3106859 words, kee

Exploring model results

In [30]:
model.wv.doesnt_match(['man', 'woman', 'child', 'kitchen'])

/Users/Hovanes/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:730: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

Looks like the model knows that men, women, and children are more closely related than kitchens

In [6]:
model.wv.most_similar('man')

[('woman', 0.6192024350166321),
 ('lady', 0.5974243879318237),
 ('lad', 0.5832437872886658),
 ('soldier', 0.537463903427124),
 ('monk', 0.523860514163971),
 ('guy', 0.5204955339431763),
 ('businessman', 0.5197274684906006),
 ('farmer', 0.5150461196899414),
 ('men', 0.5107800960540771),
 ('millionaire', 0.5076751708984375)]

In [7]:
model.wv.most_similar('awful')

[('terrible', 0.7725614309310913),
 ('atrocious', 0.7263244986534119),
 ('horrible', 0.7250977754592896),
 ('dreadful', 0.7105594873428345),
 ('abysmal', 0.705592691898346),
 ('appalling', 0.6716033220291138),
 ('horrendous', 0.6705040335655212),
 ('horrid', 0.6510659456253052),
 ('lousy', 0.6157515048980713),
 ('bad', 0.5928584337234497)]

Setting the minimum word count to 40 gave me a total vocabulary of 16,488 words with 300 features each. 

In [8]:
model.wv.vectors.shape

(16488, 300)

Creating a custom function to average all of the word vectors in a given paragraph

In [9]:
def makeFeatureVec(words, model, num_features):
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    # Index2word is a list that contains the names of the words in the model's vocabulary. Converting it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    # Loop over each word in the review and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

Creating a custom function to get the average feature vector for each of the reviews

In [10]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Calculating the average feature vector for each review and returning a 2D numpy array 
    
    # Initialize a counter
    counter = 0
 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
 
    # Loop through the reviews
    for review in reviews:

       # Print a status message every 1000th review
        if counter%1000 == 0:
            print(f"Review %d of %d" % (counter, len(reviews)))
 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)

       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

Using the functions above to calculate the average feature vectors for the training and testing sets

In [11]:
train = pd.read_csv('../data/labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

In [12]:
test = pd.read_csv('../data/testData.tsv', header=0, delimiter='\t', quoting=3)

In [13]:
def review_to_wordlist(review, remove_stopwords=False):
    
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)

    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    # 5. Return a list of words
    return(words)

In [14]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print("Creating average feature vectors for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000


/Users/Hovanes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vectors for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Revi

Pickling the cleaned training and test reviews for later use in future notebooks

In [15]:
with open('../data/cleantrainreviews.txt', 'wb') as fp:
    pickle.dump(clean_train_reviews, fp)

In [16]:
with open('../data/cleantestreviews.txt', 'wb') as fp:
    pickle.dump(clean_test_reviews, fp)

---

Now to try out some baseline models, I'll start off with RandomForestClassifier.

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB

In [18]:
X_train, X_test, y_train, y_test = train_test_split(trainDataVecs, train.sentiment, random_state=42)

In [19]:
rf = RandomForestClassifier(n_estimators=100)

Fitting a Random Forest to the labeled training data.

In [20]:
rfmodel = rf.fit(X_train, y_train)

In [21]:
rfmodel.score(X_train, y_train)

1.0

In [22]:
rfmodel.score(X_test, y_test)

0.83488

In [23]:
y_pred = rfmodel.predict(X_test)

In [24]:
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))

F1-score: 0.8393023980068514


In [25]:
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

Confusion matrix:


array([[2523,  463],
       [ 569, 2695]])

Of course, RandomForest overfits on the training data, but doesn't do too bad on the test data. It is obviously much better than chance, but there is still a lot of room for improvement.

In [26]:
rfmodel2 = rf.fit(trainDataVecs, train.sentiment)

In [27]:
result = rfmodel2.predict(testDataVecs)

In [28]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})

In [29]:
output.to_csv('../data/Word2Vec_AverageVectors.csv', index=False, quoting=3)

Now to try out a Naive Bayes Multinomial Classifier

In [32]:
mnb = MultinomialNB()

In [33]:
mnb.fit(X_train, y_train)

ValueError: Input X must be non-negative

In [35]:
X_train.shape

(18750, 300)